# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 

import matplotlib.pyplot as plt

import os

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,molchanovo,57.5817,83.7692,-14.62,96,100,2.34,RU,1708815931
1,1,aasiaat,68.7098,-52.8699,-18.62,85,94,4.57,GL,1708815931
2,2,hammerfest,70.6634,23.6821,3.25,70,80,8.75,NO,1708815931
3,3,katsuren-haebaru,26.3369,127.8719,17.27,78,100,6.17,JP,1708815931
4,4,grytviken,-54.2811,-36.5092,0.75,70,26,5.41,GS,1708815932


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

In [23]:
print(hvplot.__version__)


0.9.2


In [24]:
import pandas as pd
from pandas.core.strings import StringMethods

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud

# Drop any rows with null values
city_ideal = city_cloud.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,broken hill,-31.9500,141.4333,23.98,21,0,1.69,AU,1708815947
114,114,port orange,29.1383,-80.9956,21.12,31,0,4.12,US,1708815800
125,125,el granada,37.5027,-122.4694,24.40,53,0,3.09,US,1708815972
193,193,tallulah,32.4085,-91.1868,21.03,23,0,4.12,US,1708815994
347,347,west livingston,30.6983,-95.0019,26.03,36,0,2.68,US,1708816039
376,376,taoudenni,22.6783,-3.9836,23.68,22,0,2.89,ML,1708816048
388,388,vilhena,-12.7406,-60.1458,24.12,88,0,2.06,BR,1708815984
490,490,jeddah,21.5169,39.2192,24.92,65,0,3.09,SA,1708816102
575,575,cockburn town,21.4612,-71.1419,24.78,70,0,2.59,TC,1708816127
578,578,la romana,18.4273,-68.9728,23.44,80,0,2.86,DO,1708816128


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,broken hill,AU,-31.9500,141.4333,21,
114,port orange,US,29.1383,-80.9956,31,
125,el granada,US,37.5027,-122.4694,53,
193,tallulah,US,32.4085,-91.1868,23,
347,west livingston,US,30.6983,-95.0019,36,
376,taoudenni,ML,22.6783,-3.9836,22,
388,vilhena,BR,-12.7406,-60.1458,88,
490,jeddah,SA,21.5169,39.2192,65,
575,cockburn town,TC,21.4612,-71.1419,70,
578,la romana,DO,18.4273,-68.9728,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


broken hill - nearest hotel: Ibis Styles
port orange - nearest hotel: Sand Castle Motel
el granada - nearest hotel: Beach House
tallulah - nearest hotel: No hotel found
west livingston - nearest hotel: Best Western Plus
taoudenni - nearest hotel: No hotel found
vilhena - nearest hotel: No hotel found
jeddah - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
la romana - nearest hotel: Hotel Frano


,City,Country,Lat,Lng,Humidity,Hotel Name
48,broken hill,AU,-31.9500,141.4333,21,Ibis Styles
114,port orange,US,29.1383,-80.9956,31,Sand Castle Motel
125,el granada,US,37.5027,-122.4694,53,Beach House
193,tallulah,US,32.4085,-91.1868,23,No hotel found
347,west livingston,US,30.6983,-95.0019,36,Best Western Plus
376,taoudenni,ML,22.6783,-3.9836,22,No hotel found
388,vilhena,BR,-12.7406,-60.1458,88,No hotel found
490,jeddah,SA,21.5169,39.2192,65,No hotel found
575,cockburn town,TC,21.4612,-71.1419,70,The Salt Raker Inn
578,la romana,DO,18.4273,-68.9728,80,Hotel Frano


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)